# Custom Models and Training with Tensorflow

The Keras API and tf.data library is used for about 95% of the time. However, if you need custom or specific use cases, you can do them using the lower level python API from Tensorflow. This can be used for custom loss functions, metrics, layers, models, and so on. This can be useful in some cases.

## Using TF like NumPy

Tensorflow is built around <b><i>Tensors</i></b>. A tensor is very similar to a numpy ndarray: it is usually a multidimensional array, but it can also hold a scaler. These tensors are important when creating custom utilities for tensorflow.

### Tensors and Operations

In [1]:
import tensorflow as tf

2025-01-06 16:49:38.426895: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-06 16:49:38.437128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736203778.449667   11444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736203778.453342   11444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 16:49:38.465345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
t = tf.constant([[1.,2.,3.], [4.,5.,6.]]) # Matrix
t

I0000 00:00:1736203779.961249   11444 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9738 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
t.shape

TensorShape([2, 3])

In [4]:
t.dtype

tf.float32

In [5]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [6]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [7]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [8]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [9]:
## Scaler in a tensor
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

### Tensors and Numpy

In [10]:
import numpy as np

In [11]:
a = np.array([2., 4., 5.])

In [12]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [13]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [14]:
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [15]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### Type Conversions

By default TF will not perform type conversions automaticly because it can lead to problems when training models.

In [16]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: 

In [17]:
## Use tf.cast() if you really need to get around this
t2 = tf.constant(30.0, dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=32.0>

### Variables

Tensors are immutable. So if you need to change the values in a tensor, use a <b>TF Variable instead</b>

In [18]:
v = tf.Variable([[1.,2.,3.], [4.,5.,6.]])

In [19]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

A variable works just like a tensor and can use the same operations. However a variable can also be modified, unlike a tensor

In [20]:
v.assign(2 * v)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [21]:
v[0,1].assign(42)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
### Direct assignment doesn't work
v[1] = [7., 8., 9.]

TypeError: 'ResourceVariable' object does not support item assignment

### Other Data structures:
- Sparse Tensors
- Tensor Arrays
- Ragged Tensors
- String Tensors
- Sets
- Queues

## Customizing Models and Training Algorithms

### Custom Loss Functions

To make a custom loss function, just make a funciton that can take y_true and y_pred. Then have it make an output.

In [23]:
## Huber loss 
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

### Should only use TensorFlow operations to make it easier later.

In [24]:
# extra code – loads, splits and scales the California housing dataset, then
#              creates a simple Keras model

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

input_shape = X_train.shape[1:]

tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

/home/misha/Desktop/env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
## Just add the function to the loss parameter
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

In [26]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2


I0000 00:00:1736203794.085334   11509 service.cc:148] XLA service 0x77ae78019a30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736203794.085351   11509 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-01-06 16:49:54.106875: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1736203794.146654   11509 cuda_dnn.cc:529] Loaded cuDNN version 90300


205/363 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - loss: 0.9070 - mae: 1.3159  

I0000 00:00:1736203794.345920   11509 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7494 - mae: 1.1371 - val_loss: 0.3474 - val_mae: 0.6522
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2607 - mae: 0.5681 - val_loss: 0.2553 - val_mae: 0.5383


### Saving and Loading Models that contain Custom Components

In [27]:
### With custom loss saving is the same
model.save("my_model_with_a_custom_loss.keras")

In [28]:
### Loading is different, you need to map the loss to the original function
model = tf.keras.models.load_model("my_model_with_a_custom_loss.keras",
                                  custom_objects={"huber_fn":huber_fn})

In [29]:
## If you want to add extra hyperparameters in the custom loss
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [30]:
model.compile(loss=create_huber(2.0), optimizer='nadam')

In [31]:
### When you save this model, the threshold will not be saved and needs to be mapped
model = tf.keras.models.load_model("my_model_with_a_custom_loss.keras",
                                  custom_objects={"huber_fn": create_huber(2.0)})

You can solve this by creating a subclass of tf.keras.losses.Loss class and implementing its get_config() method

In [32]:
class HuberLoss(tf.keras.losses.Loss):
    "Huber Loss object class"
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_error = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold ** 2 / 2
        return tf.where(is_small_error, squared_error, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [33]:
model.compile(loss=HuberLoss(2.0), optimizer='nadam')
model.save("my_model_with_a_custom_loss.keras")
### Threshold will be saved now when loaded
model = tf.keras.models.load_model("my_model_with_a_custom_loss.keras",
                                  custom_objects={"HuberLoss": HuberLoss})

/home/misha/Desktop/env/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 11 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


### Custom Activation Functions, Initializers, Regularizers, and Constraints

In [34]:
## Equivalent to keras.activations.softplus()
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

## Equivalent to keras.initializers.glorot_normal()
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

## Equivalent to keras.regularizers.l1(0.01)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

## Equivalent to keras.constraints.nonneg()
def my_positive_weights(weights):  # Return value is just tf.nn.relu(weights)
    return tf.where(weigths < 0, tf.zeros_like(weights), weights)

In [35]:
## Add custom components to layer
layer = tf.keras.layers.Dense(1, activation=my_softplus,
                             kernel_initializer=my_glorot_initializer,
                             kernel_regularizer=my_l1_regularizer,
                             kernel_constraint=my_positive_weights)

If the custom component has a hyperparameter that needs to be saved, use subclassing to solve this problem

In [36]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))

    def get_config(self):
        return {"factor": self.factor}


### Custom Metrics

<b>The Difference Between losses and metrics</b>
- <b>Losses</b> like cross entropy are used by gradient descent to <b>train</b> a model, so they must be differentable and not have zero gradients everywhere. They are not supposed to be easily interpretable by humans.
- <b>Metrics</b> are used by humans to <b>evaluate</b> a model. They must be easily interpretable, and can be non-differentable and have zero gradients everywhere<br><br>
In most cases, a custom metrics is built the same way as a custom loss.

In [37]:
## Use the huber loss as a metric
model.compile(loss='mse', optimizer='nadam', metrics=[create_huber(2.0)])

Keras calculates the metric on each batch then outputs the mean of the metric after each epoch. However in some cases like with Precision, you don't want a mean. 

In [38]:
precision = tf.keras.metrics.Precision()
precision([0,1,1,1,0,1,0,1], [1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.800000011920929>

In [39]:
precision([0,1,0,0,1,0,1,1], [1,0,1,1,0,0,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

Precision is a <b>streaming or stateful metric</b>. This means that the metric is gradually updated batch after batch.

In [40]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [41]:
precision.variables

[<Variable path=precision/true_positives, shape=(1,), dtype=float32, value=[4.]>,
 <Variable path=precision/false_positives, shape=(1,), dtype=float32, value=[4.]>]

In [42]:
precision.reset_state() # Both variables get set to 0.0

In [43]:
### If you want to create your own custom streaming metric, create a subclass.
class HuberMetric(tf.keras.metrics.Metric):
    """
    Streaming HuberMetric
    """
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer='zeros')  ## Create variable to track sum of all Huber losses
        self.count = self.add_weight("count", initializer='zeros')  ## Create variable to track number of instances

    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Custom Layers

Occasionally you will want to build an architecture that contains an exotic layer that Tensorflow doesn't provide or you have a repetitive architecture where a particular block of layers is repeated many times and you want to just think of the blocks as one layer. For these cases, you build a custom layer.

In [44]:
#### For layers without weights just use Lambda
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

This custom layer can be used just like any other layer. <br>
To build a custom stateful layer(i.e. a layer with weigths), you need to create a subclass.

In [45]:
class MyDense(tf.keras.layers.Layer):

    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def built(self, batch_input_shape):
        self.kernel = self.add_weight(
            name='kernel', shape=[batch_input_shape[-1], self.units],
            initializer='glorot_normal')
        self.bias = self.add_weight(name='bias', shape=[self.units], initializer='zeros')
    
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units":self.units,
               'activation': tf.keras.activations.serialize(self.activation)}

In [47]:
## Custom Layer with multiple inputs (e.g. Concatenate)
class MyMultiLayer(tf.keras.layers.Layer):
    def call(self, X): ## X should be a tuple of all the inputs
        X1, X2 = X
        return X1 + X2, X1 * X2, X1 / X2

If your layer needs a different behavior during training and during testing, like Dropout or BatchNormalization, then you must add a training argument to the call() method and use it to decide what to do.

In [48]:
class MyGaussianNoise(tf.keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=False):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X
    

### Custom Models

An intro to this topic was in Chapter 10. 